In [28]:
import nltk
from nltk.corpus import sentence_polarity
import random
sentences = sentence_polarity.sents()

In [29]:
documents = [(sent, cat) for cat in sentence_polarity.categories() 
	for sent in sentence_polarity.sents(categories=cat)]
documents[0]

(['simplistic', ',', 'silly', 'and', 'tedious', '.'], 'neg')

In [30]:
random.shuffle(documents)

In [31]:
all_words_list = [word for (sent,cat) in documents for word in sent]
all_words = nltk.FreqDist(all_words_list)
word_items = all_words.most_common(2000)
word_features = [word for (word,count) in word_items]
word_features[200]

'ultimately'

In [32]:
def document_features(document, word_features):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [33]:
featuresets = [(document_features(d, word_features), c) for (d, c) in documents]

In [34]:
len(featuresets)

10662

In [35]:
train_set, test_set = featuresets[1000:], featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print (nltk.classify.accuracy(classifier, test_set))

0.723


In [36]:
import nltk
from nltk.corpus import PlaintextCorpusReader
import os
mycorpus = PlaintextCorpusReader('.', '.*\.txt')
mycorpus = mycorpus.raw('filename1.txt')
token=nltk.sent_tokenize(mycorpus)

In [37]:
t=token[:1000]

In [38]:
p=[]
n=[]
for i in t:
    temp=nltk.word_tokenize(i)
    a=classifier.classify(document_features(temp,word_features))
    if(a=='pos'):
        p.append(i)
    else:n.append(i)

        

In [44]:
print(len(p))
print(len(n))


418341
722301


In [9]:
negationwords = ['no', 'not', 'never', 'none', 'nowhere', 'nothing', 'noone', 'rather', 'hardly', 'scarcely', 'rarely', 'seldom', 'neither', 'nor']

def NOT_features(document, word_features, negationwords):
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = False
        features['contains(NOT{})'.format(word)] = False
    for i in range(0, len(document)):
        word = document[i]
        if ((i + 1) < len(document)) and ((word in negationwords) or (word.endswith("n't"))):
            i += 1
            features['contains({})'.format(word)] = (word in word_features)
    return features

In [10]:
NOT_featuresets = [(NOT_features(d, word_features, negationwords), c) for (d, c) in documents]

In [11]:
train_set1, test_set1 = NOT_featuresets[1000:], NOT_featuresets[:1000]
classifier1 = nltk.NaiveBayesClassifier.train(train_set1)
print (nltk.classify.accuracy(classifier1, test_set1))

0.585


In [83]:
import time

In [84]:
stopwords = nltk.corpus.stopwords.words('english')

In [85]:
t1=time.time()
p1=[]
n1=[]
for i in token:
    temp=nltk.word_tokenize(i)
    for j in temp:
        if j.isalpha() and j not in stopwords:
            continue
        else: temp.remove(j)
    a=classifier1.classify(NOT_features(temp,word_features,negationwords))
    if(a=='pos'):
        p1.append(i)
    else:n1.append(i)
t2=time.time()
print((t2-t1)/60)

261.9031235694885


In [26]:
print(p1)

1106600


In [87]:
print(len(n1))

34042

<h1> Bonus

In [16]:
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB 

**Running Classifier on first featureset**

In [18]:
classifierBonus=SklearnClassifier(BernoulliNB()).train(train_set)
nltk.classify.accuracy(classifierBonus,test_set)

0.738

**Running Classifier on second feature set**

In [20]:
classifierBonus=SklearnClassifier(BernoulliNB()).train(train_set1)
nltk.classify.accuracy(classifierBonus,test_set1)

0.585